In [7]:
def affine_gap_alignment(seq1, seq2, weight_match=2, weight_mismatch=-7, open_gap_penalty=-10, continue_gap_penalty=-1):
    def match(seq1, seq2, i, j):
        if seq2[i-1] == seq1[j-1]:
            return weight_match
        else:
            return weight_mismatch   
    n = len(seq2); m = len(seq1)
    Min = -float("inf")
    H_matrix = [[0] * (m+1) for i in range(n+1)]
    V_matrix = [[0] * (m+1) for i in range(n+1)]
    D_matrix = [[0] * (m+1) for i in range(n+1)]
    for i in range(n+1):
        for j in range(m+1):
            if i > 0 and j == 0:
                H_matrix[i][j] = Min
            if (j > 0 and i > 0) or (j > 0 and i == 0):
                H_matrix[i][j] = open_gap_penalty + (continue_gap_penalty * j)
            if j == 0 and i == 0:
                H_matrix[i][j] = 0
                V_matrix[i][j] = 0
                D_matrix[i][j] = 0
            if j > 0 and i == 0:
                V_matrix[i][j] = Min
            if (i > 0 and j > 0) or (i > 0 and j == 0):
                V_matrix[i][j] = open_gap_penalty + (continue_gap_penalty * i)
            if (i == 0 and j != 0) or (i != 0 and j == 0):
                D_matrix[i][j] = Min
            if i != 0 and j != 0:
                D_matrix[i][j] = 0
    for i in range(1, n+1):
        for j in range(1, m+1):
            H_matrix[i][j] = max(open_gap_penalty + continue_gap_penalty + D_matrix[i][j-1], continue_gap_penalty + H_matrix[i][j-1], open_gap_penalty + continue_gap_penalty + V_matrix[i][j-1])
            V_matrix[i][j] = max(open_gap_penalty + continue_gap_penalty + D_matrix[i-1][j], open_gap_penalty + continue_gap_penalty + H_matrix[i-1][j], continue_gap_penalty + V_matrix[i-1][j])
            D_matrix[i][j] = max(match(seq1, seq2, i, j) + D_matrix[i-1][j-1], H_matrix[i][j], V_matrix[i][j])
    Align_seq1 = ""
    Align_seq2 = ""
    while i > 0 or j > 0:
        if i > 0 and j > 0 and (D_matrix[i][j] == D_matrix[i-1][j-1] + match(seq1, seq2, i, j)):
            Align_seq1 = seq1[j-1] + Align_seq1
            Align_seq2 = seq2[i-1] + Align_seq2
            i -= 1
            j -= 1

        elif i > 0 and (D_matrix[i][j] == V_matrix[i][j]):
            Align_seq1 = "_" + Align_seq1 
            Align_seq2 = seq2[i-1] + Align_seq2 
            i -= 1
        elif j > 0 and (D_matrix[i][j] == H_matrix[i][j]):
            Align_seq1 = seq1[j-1] + Align_seq1
            Align_seq2 = "_" + Align_seq2
            j -= 1
        
    return(Align_seq1, Align_seq2)

In [11]:
print(affine_gap_alignment("fdsnfhusd", "fsiufsh", open_gap_penalty=-10, continue_gap_penalty=-1))

print(affine_gap_alignment("fdsnfhusd", "fsiufsh", open_gap_penalty=-8, continue_gap_penalty=-2))

('fdsnfhusd____', 'f______siufsh')
('fdsnfhusd', 'f_siuf_sh')
